In [2]:
import os
import json
import numpy as np
from glob import glob
from scipy.optimize import least_squares
from scipy.spatial.transform import Rotation

In [153]:
# utils
def se3_mul(Ta, Tb):
    Ra, ta = Ta[:3, :3], Ta[:3, 3]
    Rb, tb = Tb[:3, :3], Tb[:3, 3]
    T = np.eye(4)
    T[:3, :3] = Ra @ Rb
    T[:3, 3] = Ra @ tb + ta
    return T

def se3_inv(T):
    R, t = T[:3, :3], T[:3, 3]
    T_inv = np.eye(4)
    T_inv[:3, :3] = R.T
    T_inv[:3, 3] = -R.T @ t
    return T_inv
    
def se3_from_Rt(R, t):
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t.reshape(3)
    return T

def log_SO3(R):
    return Rotation.from_matrix(R).as_rotvec()

def log_se3(T):
    r = log_SO3(T[:3, :3])
    t = T[:3, 3]
    return np.hstack([r, t])

In [154]:
# main PGO function

def optimize_cam2base_PGO(
    A_list, B_list,
    R_cam2base_init = np.eye(3), t_cam2base_init=np.zeros(3),
    max_nfev = 200
):
    # 변수: X=cam2base(6), Y=target2base(6) → 총 12
    x0 = np.zeros(12)
    x0[:3]  = Rotation.from_matrix(R_cam2base_init).as_rotvec()
    x0[3:6] = t_cam2base_init
    
    # Y 초기값은 대충 A_0 * X * B_0 (합리적 초기)
    X0 = se3_from_Rt(R_cam2base_init, t_cam2base_init)
    Y0 = se3_mul(se3_mul(A_list[0], X0), B_list[0])
    x0[6:9]  = Rotation.from_matrix(Y0[:3,:3]).as_rotvec()
    x0[9:12] = Y0[:3,3]
    
    def unpack(x):
        rX, tX = x[:3], x[3:6]
        rY, tY = x[6:9], x[9:12]
        X = se3_from_Rt(Rotation.from_rotvec(rX).as_matrix(), tX)
        Y = se3_from_Rt(Rotation.from_rotvec(rY).as_matrix(), tY)
        return X, Y

    
    def residual(x):
        X, Y = unpack(x)
        Yin = se3_inv(Y)
        res = []
        for A, B in zip(A_list, B_list):
            T = se3_mul(se3_mul(Yin, A), se3_mul(X, B))
            e = log_se3(T)
            res.append(e)
            
        return np.concatenate(res)
    
    # Outlier가 있으면 'trf'+'soft_l1' 추천, 없으면 LM도 OK
    result = least_squares(residual, x0, method='trf', loss='soft_l1', f_scale=1.0, max_nfev=max_nfev)
    X_opt, Y_opt = unpack(result.x)
    return X_opt, Y_opt, result

In [155]:
MAIN_PATH = "../data/Calibration/Eye-to-Hand12//"

pose_files = sorted(glob(f"{MAIN_PATH}/poses/*.json"))
charuco_files = sorted(glob(f"{MAIN_PATH}/*_charuco.json"))

R_target2cam_list = []
t_target2cam_list = []
R_base2ee_list = []
t_base2ee_list = []

for pose_file, charuco_file in zip(pose_files, charuco_files):
    with open(pose_file, "r") as f:
        pose = json.load(f)
    with open(charuco_file, "r") as f:
        charuco = json.load(f)
        
        
    R_base2ee = np.array(pose["R_base2ee"]).reshape(3, 3)
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3, 1)
    
    r_tc = np.array(charuco["rvec_target2cam"], float).reshape(3)
    t_tc = np.array(charuco["tvec_target2cam"], float).reshape(3)
    
    r_tc = np.asarray(r_tc, float).reshape(3)
    R_tc = Rotation.from_rotvec(r_tc).as_matrix()

    R_target2cam_list.append(R_tc)
    t_target2cam_list.append(t_tc)
    R_base2ee_list.append(R_base2ee)
    t_base2ee_list.append(t_base2ee)

In [156]:
# 파일 길이 맞추기 
# N = min(len(R_target2cam_list), len(R_base2ee_list))
# R_target2cam_list = R_target2cam_list[:N]
# t_target2cam_list = t_target2cam_list[:N]
# R_base2ee_list    = R_base2ee_list[:N]
# t_base2ee_list    = t_base2ee_list[:N]

N = len(R_target2cam_list)
assert N == len(t_target2cam_list) == len(R_base2ee_list) == len(t_base2ee_list) and N >= 3

A_list = [se3_from_Rt(Ra, ta) for Ra, ta in zip(R_base2ee_list, t_base2ee_list)]
B_list = [se3_from_Rt(Ra, ta) for Ra, ta in zip(R_target2cam_list, t_target2cam_list)]

In [157]:
R_init = np.eye(3); t_init = np.zeros(3)
X_opt, Y_opt, info = optimize_cam2base_PGO(
    A_list, B_list,
    R_cam2base_init=R_init,
    t_cam2base_init=t_init,
    max_nfev=400
)

In [158]:
print("R_cam2base:\n", X_opt[:3,:3])
print("t_cam2base:", X_opt[:3,3])
print("cost:", info.cost, "nfev:", info.nfev)

R_cam2base:
 [[-0.01382641  0.99782485  0.06445457]
 [-0.98251157 -0.02552834  0.18444327]
 [ 0.1856875  -0.06077718  0.98072743]]
t_cam2base: [-0.15067404 -0.01696354  0.05434012]
cost: 0.059640886742699406 nfev: 12


In [159]:
B_list_inv = [np.linalg.inv(T) for T in B_list]  # cam->target 이었다면 target->cam으로
X1, Y1, info1 = optimize_cam2base_PGO(A_list, B_list_inv,
                                              R_cam2base_init=np.eye(3),
                                              t_cam2base_init=np.zeros(3),
                                              max_nfev=400)
print("Case1  t:", X1[:3,3], " cost:", info1.cost, " nfev:", info1.nfev)

Case1  t: [-0.10106714  0.14882583  0.29596798]  cost: 0.98171807564522  nfev: 32


In [160]:
A_list_inv = [np.linalg.inv(T) for T in A_list]  # ee->base 였다면 base->ee로
X2, Y2, info2 = optimize_cam2base_PGO(A_list_inv, B_list,
                                              R_cam2base_init=np.eye(3),
                                              t_cam2base_init=np.zeros(3),
                                              max_nfev=400)
print("Case2  t:", X2[:3,3], " cost:", info2.cost, " nfev:", info2.nfev)


Case2  t: [ 0.29439146 -0.05197523  0.47158857]  cost: 1.04568907237763  nfev: 28


In [161]:
A_list_inv = [np.linalg.inv(T) for T in A_list]
B_list_inv = [np.linalg.inv(T) for T in B_list]
X3, Y3, info3 = optimize_cam2base_PGO(A_list_inv, B_list_inv,
                                              R_cam2base_init=np.eye(3),
                                              t_cam2base_init=np.zeros(3),
                                              max_nfev=400)
print("Case3  t:", X3[:3,3], " cost:", info3.cost, " nfev:", info3.nfev)


Case3  t: [ 0.25175026 -0.03209375  0.54930591]  cost: 0.0532616185643493  nfev: 42


In [162]:
print("R_cam2base:\n", X3[:3,:3])
print("t_cam2base:", X3[:3,3])
print("cost:", info3.cost, "nfev:", info3.nfev)

R_cam2base:
 [[-3.03953622e-02  9.97616497e-01 -6.19471330e-02]
 [ 9.99535894e-01  3.04626947e-02  1.42561123e-04]
 [ 2.02929793e-03 -6.19140498e-02 -9.98079422e-01]]
t_cam2base: [ 0.25175026 -0.03209375  0.54930591]
cost: 0.0532616185643493 nfev: 42


In [163]:
out = {"R_cam2base": X3[:3,:3].reshape(3,3).tolist(),
       "t_cam2base": X3[:3,3].flatten().tolist()}
# cam2base_pgo_inliers.json 등으로 저장

with open(f"{MAIN_PATH}/cam2base_pgo_inliers.json", "w") as f:
    json.dump(out, f, indent=2)

### Case4: Case3 + table_normal_fix R 초기값
- robot pose: base2ee가 실제로는 ee2base
- chruco pose: cam2target이 실제로는 target2cam이었음. (case3로 확인 완료)
- 따라서 두 리스트 모두 inv()적용이 필수!
- R은 초기값이 중요하기 때문에, table_normal_fix를 활용.

In [164]:
init_cam2base_path = "../data/Calibration/Eye-to-Hand11/cam2base_table_normal_fix.json"

with open(init_cam2base_path, "r") as f:
    init_cam2base = json.load(f)

R_cam2base_init = np.array(init_cam2base["R_cam2base"]).reshape(3, 3)
t_cam2base_init = np.array(init_cam2base["t_cam2base"]).reshape(3, 1)

In [165]:
A_list_inv = [np.linalg.inv(T) for T in A_list]
B_list_inv = [np.linalg.inv(T) for T in B_list]
X4, Y4, info4 = optimize_cam2base_PGO(A_list_inv, B_list_inv,
                                    R_cam2base_init=R_cam2base_init,
                                    t_cam2base_init=np.zeros(3),
                                    max_nfev=400)
print("Case4  t:", X4[:3,3], " cost:", info4.cost, " nfev:", info4.nfev)

Case4  t: [ 0.25175025 -0.03209376  0.54930592]  cost: 0.05326161856439815  nfev: 6


- 프레임별 잔차를 확인해 outlier가 있는지 확인
- 특정 프레임이 유독 크다면 해당 프레임을 제회하고 다시 최적화하여 cost를 낮출 수 있음. 
- PGO 이후 다시 table_normal_fix를 사용하지 말것

In [166]:
# 프레임별 잔차 확인
def per_frame_residuals(X, Y, A_list, B_list):
    Yin = np.linalg.inv(Y)
    deg, mm = [], []
    for A,B in zip(A_list, B_list):
        T = np.linalg.inv(Y) @ A @ X @ B
        r = Rotation.from_matrix(T[:3,:3]).as_rotvec()
        t = T[:3,3]
        deg.append(np.linalg.norm(r)*180/np.pi)
        mm.append(np.linalg.norm(t)*1000)
    return np.array(deg), np.array(mm)

deg, mm = per_frame_residuals(X4, Y4, A_list_inv, B_list_inv)
print(f"rot err mean={deg.mean():.2f}°  max={deg.max():.2f}°")
print(f"trans err mean={mm.mean():.1f}mm  max={mm.max():.1f}mm")


rot err mean=3.09°  max=9.06°
trans err mean=8.4mm  max=26.4mm


rot err mean 7.67° / max 119.7° → 일부 프레임이 완전히 다른 좌표 정의/타임스탬프이거나 보드 포즈가 뒤집힌(≈180°) 케이스가 섞여있다는 신호.

trans err mean 26 mm / max 280 mm → outlier 몇 장이 전체 해를 당기고 있음

### MAD 기반 프레임 자동 필터

In [167]:
def per_frame_residuals_SE3(X, Y, A_list, B_list):
    deg_list, mm_list = [], []
    Tinvs = np.linalg.inv(Y)
    for A,B in zip(A_list, B_list):
        T = Tinvs @ A @ X @ B
        r = Rotation.from_matrix(T[:3,:3]).as_rotvec()
        t = T[:3,3]
        deg_list.append(np.linalg.norm(r)*180/np.pi)
        mm_list.append(np.linalg.norm(t)*1000)
    return np.array(deg_list), np.array(mm_list)

def mad_filter(vals, k=3.5):
    med = np.median(vals)
    mad = np.median(np.abs(vals - med)) + 1e-9
    z = 0.6745 * (vals - med) / mad
    keep = np.abs(z) < k
    return keep, med, mad

In [168]:
# 1) 현재 해 기준 프레임별 잔차
deg, mm = per_frame_residuals_SE3(X4, Y4, A_list_inv, B_list_inv)
idx = np.arange(len(deg))

# 2) 회전/번역 각각 MAD 필터
keep_r, r_med, r_mad = mad_filter(deg, k=3.5)
keep_t, t_med, t_mad = mad_filter(mm,  k=3.5)
keep = keep_r & keep_t

print(f"[before] keep {keep.sum()}/{len(keep)}  rot(med±MAD)={r_med:.2f}°±{r_mad:.2f}°,  trans(med±MAD)={t_med:.1f}±{t_mad:.1f} mm")
print("worst 10 by rot:", idx[np.argsort(-deg)][:10], deg[np.argsort(-deg)][:10])
print("worst 10 by trans:", idx[np.argsort(-mm)][:10], mm[np.argsort(-mm)][:10])

[before] keep 22/25  rot(med±MAD)=2.74°±0.67°,  trans(med±MAD)=7.7±3.3 mm
worst 10 by rot: [18 21 19 11 16 13 17  9 10 23] [9.06206128 8.45455773 6.7146368  3.69528519 3.40734024 3.37393143
 3.33520922 3.27611586 3.26841405 3.07795924]
worst 10 by trans: [21 19 17 18 14 11 15 12  2  0] [26.44055491 16.962263   12.52429411 12.22190582 11.71508111 11.39125239
 11.03872797 10.59665001  9.61375907  9.39742703]


In [169]:
# 3) inlier만으로 재최적화
A_in = [A_list_inv[i] for i in idx[keep]]
B_in = [B_list_inv[i] for i in idx[keep]]

X_in, Y_in, info_in = optimize_cam2base_PGO(
    A_in,B_in,
    R_cam2base_init=np.eye(3), t_cam2base_init=np.zeros(3),
    max_nfev=2000
)

deg_in, mm_in = per_frame_residuals_SE3(X_in, Y_in, A_in, B_in)
print(f"[after ] rot mean={deg_in.mean():.2f}°, max={deg_in.max():.2f}°")
print(f"[after ] trans mean={mm_in.mean():.1f}mm, max={mm_in.max():.1f}mm")
print("t_cam2base (inlier):", X_in[:3,3])

[after ] rot mean=2.27°, max=3.51°
[after ] trans mean=6.5mm, max=11.0mm
t_cam2base (inlier): [ 0.24881543 -0.0296964   0.55479992]


In [170]:
out = {"R_cam2base": X_in[:3,:3].reshape(3,3).tolist(),
       "t_cam2base": X_in[:3,3].flatten().tolist()}
# cam2base_pgo_inliers.json 등으로 저장

with open(f"{MAIN_PATH}/cam2base_pgo_inliers.json", "w") as f:
    json.dump(out, f, indent=2)

# Depth를 추가하여 잔여 오차 수정
- 실험결과 depth미사용시와 큰 차이가 없으므로, 코드는 유지하지만 사용은 않기로 함.

In [171]:
# NEW: depth 평면 팩터 포함 PGO
def optimize_cam2base_with_depth(
    A_list, B_list,                      # 4x4 SE(3) lists (정의는 A=base->ee, B=target->camRGB 로 맞춘 것)
    depth_points_list,                   # len=N, 각 원소 (Ki x 3) in DEPTH camera frame (테이블 inlier만)
    n_b=np.array([0,0,1.0]), d_b=0.0,    # base 평면 (z=0이면 [0,0,1], d=0)
    R_cam2base_init=np.eye(3), t_cam2base_init=np.zeros(3),
    R_depth2rgb_init=np.eye(3), t_depth2rgb_init=np.zeros(3),
    lambda_init=0.0,                     # depth scale 보정 (p_d'=(1+λ) p_d)
    w_axxb=1.0, w_plane=3.0,             # 두 팩터 가중치
    loss='soft_l1', f_scale=0.5, max_nfev=2000
):
    from scipy.optimize import least_squares
    from scipy.spatial.transform import Rotation as Rotation
    import numpy as np

    N = len(A_list)
    assert N == len(B_list) == len(depth_points_list) and N >= 3

    # ---- 초기값: X=camRGB->base, Y=target->base, E=depth->rgb, λ ----
    rX0 = Rotation.from_matrix(R_cam2base_init).as_rotvec(); tX0 = np.asarray(t_cam2base_init).reshape(3)
    X0  = se3_from_Rt(R_cam2base_init, tX0)
    Y0  = se3_mul(se3_mul(A_list[0], X0), B_list[0])
    rY0 = Rotation.from_matrix(Y0[:3,:3]).as_rotvec();      tY0 = Y0[:3,3]
    rE0 = Rotation.from_matrix(R_depth2rgb_init).as_rotvec(); tE0 = np.asarray(t_depth2rgb_init).reshape(3)
    lam0 = np.array([lambda_init], float)

    x0 = np.hstack([rX0, tX0, rY0, tY0, rE0, tE0, lam0])  # 19차원

    n_b = np.asarray(n_b, float).reshape(3)
    n_b = n_b / (np.linalg.norm(n_b)+1e-12)
    d_b = float(d_b)

    def unpack(x):
        rX, tX = x[0:3],   x[3:6]
        rY, tY = x[6:9],   x[9:12]
        rE, tE = x[12:15], x[15:18]
        lam    = x[18]
        X = se3_from_Rt(Rotation.from_rotvec(rX).as_matrix(), tX)
        Y = se3_from_Rt(Rotation.from_rotvec(rY).as_matrix(), tY)
        E = se3_from_Rt(Rotation.from_rotvec(rE).as_matrix(), tE)
        return X, Y, E, lam

    def residual(x):
        X, Y, E, lam = unpack(x)
        Yin = se3_inv(Y)
        res = []

        # (1) Hand–Eye: Y^{-1} A X B ≈ I  → 6N잔차
        for A,B in zip(A_list, B_list):
            T = se3_mul(se3_mul(Yin, A), se3_mul(X, B))
            e = log_se3(T)
            res.append(np.sqrt(w_axxb)*e)

        # (2) Depth 평면 잔차:  r = n_b^T ( X * ( E * ((1+λ)*p_d) ) ) + d_b
        for Pd in depth_points_list:
            if Pd is None or len(Pd)==0: 
                continue
            P = np.asarray(Pd, float).reshape(-1,3)            # (K,3) in depth
            P = (1.0 + lam) * P                                # scale 보정
            # depth->rgb
            Pr = (E[:3,:3] @ P.T) + E[:3,3:4]                  # (3,K)
            # rgb->base (camRGB==X의 source)
            Pb = (X[:3,:3] @ Pr) + X[:3,3:4]                   # (3,K)
            r  = (n_b @ Pb + d_b).ravel()                      # (K,)
            res.append(np.sqrt(w_plane)*r)

        return np.hstack(res)

    # solve
    result = least_squares(residual, x0, method='trf', loss=loss, f_scale=f_scale, max_nfev=max_nfev)
    X_opt, Y_opt, E_opt, lam_opt = unpack(result.x)
    return X_opt, Y_opt, E_opt, lam_opt, result


In [172]:
# depth to point
# NEW: depth 픽셀 → depth카메라 3D (m)
def depth_to_points(depth_m, K, mask=None):
    # depth_m: (H,W) in meters
    fx, fy, cx, cy = K
    if mask is None:
        mask = depth_m > 0
    ys, xs = np.nonzero(mask)
    z = depth_m[ys, xs]
    x = (xs - cx) * z / fx
    y = (ys - cy) * z / fy
    return np.stack([x,y,z], axis=1)  # (N,3)

# NEW: RANSAC 평면 inlier (테이블만 추리기)
def ransac_plane(P, thr=0.003, iters=200, seed=0):
    if len(P) < 50: return P
    rng = np.random.default_rng(seed)
    best = []
    for _ in range(iters):
        idx = rng.choice(len(P), 3, replace=False)
        p1,p2,p3 = P[idx]
        n = np.cross(p2-p1, p3-p1)
        nrm = np.linalg.norm(n)
        if nrm < 1e-8: continue
        n /= nrm
        d = -n @ p1
        dist = np.abs(P @ n + d)
        inl = np.where(dist < thr)[0]
        if len(inl) > len(best): best = inl
    return P[best]


In [173]:
import open3d as o3d

def segment_plane_from_depth(depth_m, K, roi=None, dist_thresh=0.003, iters=300):
    """
    depth_m : (H,W) float [meter]
    K       : (fx, fy, cx, cy)
    roi     : (y0,y1,x0,x1) 영역만 사용 (테이블 고정이면 속도↑/강건↑)
    return  : mask (H,W, bool), plane (a,b,c,d) with ||[a,b,c]||=1 in camera frame
    """
    fx, fy, cx, cy = K
    H, W = depth_m.shape

    # 1) ROI 선택 + 유효 깊이
    if roi is None:
        y0,y1,x0,x1 = 0, H, 0, W
    else:
        y0,y1,x0,x1 = roi
    D = depth_m[y0:y1, x0:x1]
    valid = np.isfinite(D) & (D > 0)

    if valid.sum() < 50:
        mask = np.zeros_like(depth_m, dtype=bool)
        return mask, None

    # 2) depth → point cloud (camera/depth 좌표)
    ys, xs = np.nonzero(valid)
    z = D[ys, xs]
    x = (xs + x0 - cx) * z / fx
    y = (ys + y0 - cy) * z / fy
    pts = np.stack([x, y, z], axis=1)

    # 3) Open3D plane segmentation
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))
    plane_model, inliers = pcd.segment_plane(distance_threshold=dist_thresh,
                                             ransac_n=3,
                                             num_iterations=iters)
    # plane: ax+by+cz+d=0 (in camera frame), normal normalized by open3d
    a,b,c,d = plane_model
    mask = np.zeros_like(depth_m, dtype=bool)
    # 인라이어 인덱스를 원래 이미지 좌표로 되돌리기
    mask_idx = (ys + y0, xs + x0)
    sel = np.zeros_like(ys, dtype=bool)
    sel[inliers] = True
    mask[mask_idx] = sel
    return mask, (a,b,c,d)

In [175]:
import numpy as np
from glob import glob

# 0) 입력
depth_paths = sorted(glob(f"{MAIN_PATH}/depth/*.npy"))
# 카메라 내참수 
Kd = (fx, fy, cx, cy) = (430.341, 430.341, 422.634, 244.632)

# 테이블이 항상 하단 가운데라 가정한 ROI 예시(이미지 크기에 맞게 수정)
roi = None

# 1) 각 프레임에서 테이블 인라이어 3D점 만들기
depth_points_list = []
for dp in depth_paths:
    depth_m = np.load(dp).astype(np.float32)*0.001   # (H,W), meter
    # 자동 평면 분할 (Open3D or Numpy 버전 선택)
    mask, plane_cam = segment_plane_from_depth(depth_m, Kd, roi=roi,
                                               dist_thresh=0.003, iters=300)
    # 테이블 픽셀만 3D로
    ys, xs = np.nonzero(mask)
    z = depth_m[ys, xs]
    x = (xs - cx) * z / fx
    y = (ys - cy) * z / fy
    P = np.stack([x,y,z], axis=1)     # (N,3) depth(camera) frame

    # 너무 많으면 샘플링(속도/수렴 안정 위해)
    if len(P) > 2000:
        step = len(P) // 1000
        P = P[::step]
    depth_points_list.append(P)


In [176]:
depth_points_in = [depth_points_list[i] for i in idx[keep]]

R_init = R_cam2base_init      # table_normal_fix의 R 추천
t_init = np.zeros(3)
R_d2r_init = np.eye(3); t_d2r_init = np.zeros(3)

# 최적화
X_opt, Y_opt, E_opt, lam_opt, info = optimize_cam2base_with_depth(
    A_in, B_in,
    depth_points_list=depth_points_in,
    n_b=np.array([0,0,1.0]), d_b=0.0,      # base의 테이블 평면이 z=0일 때
    R_cam2base_init=R_init, t_cam2base_init=t_init,
    R_depth2rgb_init=R_d2r_init, t_depth2rgb_init=t_d2r_init,
    lambda_init=0.0,
    w_axxb=1.0, w_plane=3.0,               # 평면 가중 2~5 범위로 튜닝
    loss='soft_l1', f_scale=0.5, max_nfev=2000
)

print("t_cam2base:", X_opt[:3,3])
print("lambda (depth scale):", lam_opt)
print("T_depth->rgb:\n", E_opt)
print("cost:", info.cost, " nfev:", info.nfev)

# 핸드아이 잔차(기존 함수 재사용)
deg_in, mm_in = per_frame_residuals_SE3(X_opt, Y_opt, A_in, B_in)
print(f"[after depth] rot mean={deg_in.mean():.2f}°, max={deg_in.max():.2f}°")
print(f"[after depth] trans mean={mm_in.mean():.1f}mm, max={mm_in.max():.1f}mm")


t_cam2base: [ 0.24881023 -0.02969705  0.55480214]
lambda (depth scale): -0.9999999999769016
T_depth->rgb:
 [[ 0.01735516  0.99894172  0.04259379 -0.3529471 ]
 [-0.99961228  0.01826299 -0.02101778  0.01946257]
 [-0.02177343 -0.04221251  0.99887137  0.56383583]
 [ 0.          0.          0.          1.        ]]
cost: 0.019474820520932745  nfev: 10
[after depth] rot mean=2.27°, max=3.51°
[after depth] trans mean=6.5mm, max=11.0mm


In [177]:
out = {"R_cam2base": X_opt[:3,:3].reshape(3,3).tolist(),
       "t_cam2base": X_opt[:3,3].flatten().tolist()}
# cam2base_pgo_inliers.json 등으로 저장

with open(f"{MAIN_PATH}/cam2base_pgo_inliers_depth.json", "w") as f:
    json.dump(out, f, indent=2)

In [148]:
# ----- 깊이 진단 ----- #
print("frames:", len(depth_points_in))
print("pts/frame:", [len(P) if P is not None else 0 for P in depth_points_in])
print("total pts:", sum(len(P) for P in depth_points_in if P is not None))

frames: 49
pts/frame: [1016, 1013, 1006, 1005, 1012, 1005, 1003, 1006, 1001, 1003, 1001, 1007, 1002, 1002, 1001, 1003, 1004, 1004, 1006, 1003, 1001, 1006, 1003, 1003, 1005, 1001, 1006, 1006, 1003, 1001, 1003, 1002, 1005, 1002, 1001, 1007, 1001, 1004, 1004, 1003, 1006, 1003, 1004, 1008, 1001, 1002, 1010, 1001, 1002]
total pts: 49207


In [178]:
def plane_residual_stats(X, E, n_b, d_b, depth_points_list):
    import numpy as np
    all_r = []
    for Pd in depth_points_list:
        if Pd is None or len(Pd)==0: continue
        Pr = (E[:3,:3] @ Pd.T) + E[:3,3:4]
        Pb = (X[:3,:3] @ Pr) + X[:3,3:4]
        r  = (n_b @ Pb + d_b).ravel()
        all_r.append(r)
    r = np.concatenate(all_r) if all_r else np.array([])
    if r.size:
        print(f"plane resid: mean={r.mean()*1000:.2f} mm, std={r.std()*1000:.2f} mm, max={np.max(np.abs(r))*1000:.2f} mm")
    else:
        print("plane resid: (no points)")
        
plane_residual_stats(
    X_opt,
    E_opt,
    np.array([0,0,1.0]),  # n_b
    0.0,                  # d_b
    depth_points_in       # 각 프레임 테이블 포인트 리스트
)

plane resid: mean=-535.98 mm, std=17.45 mm, max=584.26 mm


In [179]:
def fit_plane_in_base(X, E, depth_points_list, sample=20000, seed=0):
    import numpy as np
    rng = np.random.default_rng(seed)
    pts = []
    for Pd in depth_points_list:
        if Pd is None or len(Pd)==0: continue
        # depth->rgb
        Pr = (E[:3,:3] @ Pd.T) + E[:3,3:4]
        # rgb->base
        Pb = (X[:3,:3] @ Pr) + X[:3,3:4]
        pts.append(Pb.T)
    if not pts: return None, None
    P = np.vstack(pts)
    if len(P) > sample:
        idx = rng.choice(len(P), sample, replace=False)
        P = P[idx]
    # 평면 n,d (ax+by+cz+d=0), ||n||=1
    # SVD로 n 추정
    Pc = P - P.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Pc)
    n = Vt[-1]                   # 가장 작은 고유벡터 ~ 법선
    n = n / (np.linalg.norm(n)+1e-12)
    d = -n @ P.mean(axis=0)
    return n, d

# 사용 예
n_b_est, d_b_est = fit_plane_in_base(X_opt, E_opt, depth_points_in)
print("estimated base-plane:", n_b_est, d_b_est)
# n_b_est가 거의 [0,0,±1]이면 테이블이 base z축과 정렬. d_b_est가 z=0이면 0 근처.


estimated base-plane: [-0.07224822  0.04481629 -0.99637929] -0.5127711577176425


In [180]:
# bounds가 들어간 버전 (핵심만)
def optimize_cam2base_with_depth_bounded(
    A_list, B_list, depth_points_list, n_b, d_b,
    R_cam2base_init, t_cam2base_init,
    R_depth2rgb_init, t_depth2rgb_init,
    w_axxb=1.0, w_plane=2.0, f_scale=0.5, max_nfev=2000
):
    from scipy.optimize import least_squares
    from scipy.spatial.transform import Rotation as Rotation
    import numpy as np

    # x = [rX(3) tX(3) rY(3) tY(3) rE(3) tE(3)]  ← λ 제외
    rX0 = Rotation.from_matrix(R_cam2base_init).as_rotvec(); tX0 = np.zeros(3)
    X0  = se3_from_Rt(R_cam2base_init, tX0)
    Y0  = se3_mul(se3_mul(A_list[0], X0), B_list[0])
    rY0 = Rotation.from_matrix(Y0[:3,:3]).as_rotvec(); tY0 = Y0[:3,3]
    rE0 = Rotation.from_matrix(R_depth2rgb_init).as_rotvec(); tE0 = t_depth2rgb_init

    x0 = np.hstack([rX0, tX0, rY0, tY0, rE0, tE0])

    def unpack(x):
        rX,tX = x[0:3], x[3:6]
        rY,tY = x[6:9], x[9:12]
        rE,tE = x[12:15], x[15:18]
        X = se3_from_Rt(Rotation.from_rotvec(rX).as_matrix(), tX)
        Y = se3_from_Rt(Rotation.from_rotvec(rY).as_matrix(), tY)
        E = se3_from_Rt(Rotation.from_rotvec(rE).as_matrix(), tE)
        return X,Y,E

    def residual(x):
        X,Y,E = unpack(x)
        Yin = se3_inv(Y)
        res = []
        # AX=XB
        for A,B in zip(A_list, B_list):
            T = se3_mul(se3_mul(Yin, A), se3_mul(X, B))
            res.append(log_se3(T))
        # plane (lam=0)
        for Pd in depth_points_list:
            if Pd is None or len(Pd)==0: continue
            Pr = (E[:3,:3] @ Pd.T) + E[:3,3:4]
            Pb = (X[:3,:3] @ Pr) + X[:3,3:4]
            r  = (n_b @ Pb + d_b).ravel()
            res.append(np.sqrt(w_plane)*r)
            
        # E prior (작게 움직이도록)
        rX,tX,rY,tY,rE,tE = x[0:3],x[3:6],x[6:9],x[9:12],x[12:15],x[15:18]
        res.append(0.1 * rE)          # ~5–6°
        res.append(100.0 * tE)        # 1 cm -> residual=1
        return np.hstack(res)

    # 각 성분 bounds (rE는 성분별 ±0.1rad≈5.7°, tE는 ±0.03m)
    lo = np.array([-np.inf]*12 + [-0.1, -0.1, -0.1, -0.03, -0.03, -0.03])
    hi = np.array([ np.inf]*12 + [ 0.1,  0.1,  0.1,  0.03,  0.03,  0.03])

    result = least_squares(residual, x0, method='trf',
                           loss='soft_l1', f_scale=f_scale,
                           bounds=(lo,hi), max_nfev=max_nfev)
    return *unpack(result.x), result


Xb, Yb, Eb, info = optimize_cam2base_with_depth_bounded(
    A_in, B_in, depth_points_in,
    n_b=np.array([0,0,1.0]), d_b=0.0,
    R_cam2base_init=R_cam2base_init, t_cam2base_init=np.zeros(3),
    R_depth2rgb_init=np.eye(3), t_depth2rgb_init=np.zeros(3),
    w_axxb=1.0, w_plane=2.0, max_nfev=2000
)

print("t_cam2base:", Xb[:3,3])
print("T_depth->rgb:\n", Eb)
print("cost:", info.cost, " nfev:", info.nfev)

# 핸드아이 잔차(기존 함수 재사용)
deg_in, mm_in = per_frame_residuals_SE3(Xb, Yb, A_in, B_in)
print(f"[after depth] rot mean={deg_in.mean():.2f}°, max={deg_in.max():.2f}°")
print(f"[after depth] trans mean={mm_in.mean():.1f}mm, max={mm_in.max():.1f}mm")


t_cam2base: [ 0.2471848  -0.03016809  0.54781825]
T_depth->rgb:
 [[ 9.99733772e-01  3.26958845e-04  2.30711367e-02  2.03233327e-08]
 [-8.39147118e-04  9.99753326e-01  2.21942042e-02  3.96312622e-08]
 [-2.30581891e-02 -2.22076556e-02  9.99487439e-01  7.94116743e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
cost: 0.05117989737430939  nfev: 12
[after depth] rot mean=2.27°, max=3.51°
[after depth] trans mean=6.7mm, max=11.2mm


In [181]:
out = {"R_cam2base": Xb[:3,:3].reshape(3,3).tolist(),
       "t_cam2base": Xb[:3,3].flatten().tolist()}
# cam2base_pgo_inliers.json 등으로 저장

with open(f"{MAIN_PATH}/cam2base_pgo_inliers_depth_bounded.json", "w") as f:
    json.dump(out, f, indent=2)